# Import del dataset

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#importing dataset
australia = pd.read_csv('path.../AUS.csv', header = 0)
#australia.head(5)

# Rinominiamo i campi

In [3]:
new_names = {'Date': 'date', 'Location': 'location','MinTemp': 'min_temp', 
            'MaxTemp': 'max_temp', 'Rainfall':'rainfall', 'Evaporation': 'evaporation', 'Sunshine': 'sunshine', 
            'WindGustDir':'wind_gust_dir', 'WindGustSpeed': 'wind_gust_speed', 'WindDir9am': 'wind_dir_9am', 'WindDir3pm': 'wind_dir_3pm',
            'WindSpeed9am': 'wind_speed_9am', 'WindSpeed3pm': 'wind_speed_3pm',
            'Humidity9am':'humidity_9am', 'Humidity3pm':'humidity_3pm', 'Pressure9am':'pressure_9am', 'Pressure3pm':'pressure_3pm',
            'Cloud9am':'cloud_9am', 'Cloud3pm':'cloud_3pm', 'Temp9am':'temp_9am', 'Temp3pm':'temp_3pm',
            'RainToday': 'rain_today', 'RainTomorrow': 'rain_tomorrow' }
australia.rename(columns=new_names, inplace=True)


In [ ]:
print(australia.isnull().sum())

# drop delle colonne sunshine, evaporation

In [4]:
australia = australia.drop(['sunshine','evaporation'], axis=1)
#australia.head(3)

In [5]:
#correzione del campo wind_dir e wind_speed: mettiamo 0 al posto di null ai campi dir se la velocità del vento è 0 (non c'è vento in quel momento)
australia['wind_dir_9am'] = australia[['wind_dir_9am','wind_speed_9am']].apply(lambda x: '0' if x['wind_speed_9am']==0 else x['wind_dir_9am'], axis=1)
australia['wind_dir_3pm'] = australia[['wind_dir_3pm','wind_speed_3pm']].apply(lambda x: '0' if x['wind_speed_3pm']==0 else x['wind_dir_3pm'], axis=1)

# rimuoviamo i gust perchè sono presi dalle colonne sopra, tra il massimo della giornata
australia = australia.drop(['wind_gust_dir','wind_gust_speed'], axis=1)

In [ ]:
print(australia.isnull().sum())

In [6]:
#togliamo tutte quelle città che hanno almeno il 40% di null sul campo cloud3pm
location = australia.location.unique()
#print(location)
count = 0
for loc in location:
    #print(loc, len(australia[australia['location']==loc]))
    lung = len(australia[australia['location']==loc])
    if australia[australia['location']==loc].isnull().sum().cloud_3pm > lung * 0.4:
        australia = australia.drop(australia[australia['location'] == loc].index)
        count +=1
#print(count)


18


In [ ]:
print(australia.isnull().sum())
print(len(australia))
#australia.to_csv('C:/Users/angel/OneDrive/Desktop/ post_etl.csv', index = False)

In [7]:
#rimuoviamo la riga che ha meno di 16 campi non nulli
australia = australia.dropna(thresh = 16)

#print(australia.isnull().sum())
#print(len(australia))

In [8]:
#imputation dei valori, andando a prendere il valore di quel campo alla riga precedente
australia = australia.ffill(axis=0)
australia.head()

,date,location,min_temp,max_temp,rainfall,wind_dir_9am,wind_dir_3pm,wind_speed_9am,wind_speed_3pm,humidity_9am,humidity_3pm,pressure_9am,pressure_3pm,cloud_9am,cloud_3pm,temp_9am,temp_3pm,rain_today,rain_tomorrow
6049,2009-01-01,Cobar,17.9,35.2,0.0,ENE,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
6050,2009-01-02,Cobar,18.4,28.9,0.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
6052,2009-01-04,Cobar,19.4,37.6,0.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
6053,2009-01-05,Cobar,21.9,38.4,0.0,WNW,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No
6054,2009-01-06,Cobar,24.2,41.0,0.0,NW,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,No,No


In [ ]:
print(australia.isnull().sum())

In [9]:
#aggiungiamo la colonna state in funzione della città 

#stati:
sa = ['Adelaide', 'MountGambier', 'Nuriootpa', 'Woomera'] #australia meridionale
wa = ['Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport', 'Perth', 'SalmonGums',
      'Walpole'] #australia occidentale
nsw = ['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'Penrith', 'Richmond', 'Sydney', 'SydneyAirport',
       'WaggaWagga', 'Williamtown', 'Wollongong'] #Nuovo Galles del Sud
qld = ['Brisbane', 'Cairns', 'GoldCoast', 'Townsville'] #Queensland
tas = ['Hobart', 'Launceston'] #Tasmania
vic = ['Ballarat', 'Bendigo', 'Sale', 'Melbourne', 'MelbourneAirport',
      'Mildura', 'Nhil', 'Portland', 'Watsonia', 'Dartmoor'] #Victoria
act = ['Camberra', 'Tuggeranong', 'MountGinini'] #territorio della capitale australiana
nt = [ 'AliceSprings', 'Darwin', 'Katherine', 'Uluru'] #Territorio del nord
nfk = ['NorfolkIsland'] #territorio esterno del norfolk
 
#gli ultimi quattro si potrebbero unire anche in un unico territorio del sud est australiano,
#a livello di clima dovrebbero essere molto simii potrebbe avere difficoltà a classificarli

def State(a):
    if a in sa:
        return 'SA'
    elif a in wa:
        return 'WA'
    elif a in nsw:
        return 'NSW'
    elif a in qld:
        return 'QLD'
    elif a in tas:
        return 'TAS'
    elif a in vic:
        return 'VIC'
    elif a in act:
        return 'ACT'
    elif a in nt:
        return 'NT'
    elif a in nfk:
        return 'NFK'
    

#le = preprocessing.LabelEncoder()
#crime['category_id'] = le.fit_transform(australia['state'])

australia['state'] = australia.location.apply(lambda x: State(x))


#australia[australia['rainfall'] > 100].head(20)

In [15]:
australia.head()

,date,location,min_temp,max_temp,rainfall,wind_dir_9am,wind_dir_3pm,wind_speed_9am,wind_speed_3pm,humidity_9am,humidity_3pm,pressure_9am,pressure_3pm,cloud_9am,cloud_3pm,temp_9am,temp_3pm,rain_today,rain_tomorrow,state
6049,2009-01-01,Cobar,17.9,35.2,0.0,ENE,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,No,3.0
6050,2009-01-02,Cobar,18.4,28.9,0.0,SSE,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,No,3.0
6052,2009-01-04,Cobar,19.4,37.6,0.0,NNE,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,No,3.0
6053,2009-01-05,Cobar,21.9,38.4,0.0,WNW,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,No,3.0
6054,2009-01-06,Cobar,24.2,41.0,0.0,NW,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,No,3.0
